### Customer Lifetime Value prediction

In [1277]:
import pandas as pd

In [1278]:
df = pd.read_csv(filepath_or_buffer="./data/customer_segmentation.csv",
                 encoding="cp1252")  # Specify the encoding!
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


#### Convert the date field

In [1279]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

#### Create a new field / feature from existing ones

In [1280]:
df['InvoiceYearMonth'] = df['InvoiceDate'].map(lambda date: 100 * date.year + date.month)
df['Revenue'] = df['UnitPrice'] * df['Quantity']  # This should be predicted!

In [1281]:
df.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYearMonth,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,201012,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,201012,20.34


In [1282]:
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID,InvoiceYearMonth,Revenue
count,541909.000000,541909,541909.000000,406829.000000,541909.000000,541909.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570,201099.713989,17.987795
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000,201012.000000,-168469.600000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000,201103.000000,3.400000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000,201107.000000,9.750000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000,201110.000000,17.400000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000,201112.000000,168469.600000
std,218.081158,NaN,96.759853,1713.600303,25.788703,378.810824


In [1283]:
filtered = df.drop(columns=['StockCode', 'Description'])
filtered = filtered.dropna()

# keep only rows that make sens, i.e. are either definitely costs or definitely profits
filtered = filtered[
    (filtered['Revenue'] > 0) & (filtered['Quantity'] > 0) | (filtered['Revenue'] < 0) & (
                filtered['Quantity'] < 0)]

max_date = df['InvoiceDate'].max()
date_limit = max_date - pd.Timedelta(days=180)
filtered = filtered[filtered['InvoiceDate'] >= date_limit]
# filtered = filtered['CustomerID'].apply(lambda x: int(x) if x == x else "")


positive_revenue = filtered[filtered['Revenue'] >= 0]
positive_revenue = positive_revenue.drop_duplicates(subset=['InvoiceNo'], keep='first')

invoice_counts = positive_revenue['CustomerID'].value_counts()

customers_with_multiple_invoices = invoice_counts[invoice_counts >= 2.0].keys()

print(len(customers_with_multiple_invoices))
filtered_aux = filtered[filtered['CustomerID'].isin(customers_with_multiple_invoices)]

print(filtered_aux.head(5))
filtered_aux.shape

2015
       InvoiceNo  Quantity         InvoiceDate  UnitPrice  CustomerID  \
223297    556479         3 2011-06-12 12:50:00       8.95     16983.0   
223298    556479         1 2011-06-12 12:50:00       1.65     16983.0   
223299    556479         1 2011-06-12 12:50:00       2.95     16983.0   
223300    556479         2 2011-06-12 12:50:00       2.95     16983.0   
223301    556479         6 2011-06-12 12:50:00       3.75     16983.0   

               Country  InvoiceYearMonth  Revenue  
223297  United Kingdom            201106    26.85  
223298  United Kingdom            201106     1.65  
223299  United Kingdom            201106     2.95  
223300  United Kingdom            201106     5.90  
223301  United Kingdom            201106    22.50  


(210890, 8)

In [1284]:
latest_buy = filtered_aux.groupby('CustomerID').InvoiceDate.max().reset_index()
latest_buy.columns = ['CustomerID','LatestBuyDate']
latest_buy['Recency'] = (max_date - latest_buy['LatestBuyDate']).dt.days
latest_buy.drop(['LatestBuyDate'], axis=1, inplace=True)


purchase_dates = filtered_aux.groupby('CustomerID')['InvoiceDate'].unique()
frequency = purchase_dates.apply(len)
average_time_between_buys = purchase_dates.apply(lambda p: [(p[n] - p[n-1]).days for n in range(1, len(p))])
average_time_between_buys = average_time_between_buys.apply(lambda p: sum(p)/len(p) if len(p) > 1 else sum(p))

print(average_time_between_buys.describe())

# print(frequency.head(5))
# print(frequency.describe())
# 
# frequency['InvoiceDate'].apply(lambda date_list: len(date_list))
# frequency = filtered_aux.groupby('CustomerID').InvoiceDate.unique().count()
# frequency.columns = ['CustomerID', 'Frequency']


revenue = filtered_aux.groupby('CustomerID').Revenue.sum().reset_index()
revenue = revenue[np.abs(revenue.Revenue - revenue.Revenue.mean()) < (3 * revenue.Revenue.std())]

final = pd.merge(revenue, frequency.to_frame(), on='CustomerID')
final = pd.merge(final, average_time_between_buys.to_frame(), on='CustomerID')

final.rename(columns={'InvoiceDate_x':'Frequency', 'InvoiceDate_y':'MeanTimeBetweenBuys'}, inplace=True)
final['Recency'] = latest_buy['Recency']


final.head(5)
final.describe()
# revenue.head(5)
# revenue.describe()


count    2015.000000
mean       35.596346
std        29.747180
min         0.000000
25%        15.763889
50%        27.000000
75%        47.000000
max       175.000000
Name: InvoiceDate, dtype: float64


,CustomerID,Revenue,Frequency,MeanTimeBetweenBuys,Recency
count,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000
mean,15266.786072,1635.085441,4.966433,35.877126,29.441884
std,1708.623415,2165.007939,5.078173,29.745633,32.256089
min,12347.000000,-229.200000,1.000000,0.000000,0.000000
25%,13813.750000,563.427500,2.000000,16.100000,7.000000
50%,15256.500000,1005.105000,4.000000,27.250000,18.000000
75%,16745.250000,1830.035000,6.000000,47.000000,40.000000
max,18287.000000,23934.000000,127.000000,175.000000,171.000000
